In [133]:
import numpy as np

def square_root(A):
    n = A.shape[0]
    U = np.zeros((n, n), dtype=np.complex128)
    
    for i in range(n):
        for j in range(i, n):
            s = sum(U[k, i].conj() * U[k, j] for k in range(i))
            
            if i == j:
                U[i, i] = np.sqrt(A[i, i] - s)
            else:
                U[i, j] = (A[i, j] - s) / U[i, i]
    return U

def solve_upper_triangular(U, b):
    n = len(b)
    x = np.zeros(n, dtype=np.complex128)
    for i in reversed(range(n)):
        x[i] = (b[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]
    return x

def solve_lower_triangular(L, b):
    n = len(b)
    x = np.zeros(n, dtype=np.complex128)
    for i in range(n):
        x[i] = (b[i] - np.dot(L[i, :i], x[:i])) / L[i, i]
    return x

A = np.array([
    [0.83, 2.18, -1.73],
    [2.18, -1.41, 1.03],
    [-1.73, 1.03, 2.27]
], dtype=np.float64)

y = np.array([0.83, -1.12, 0.47], dtype=np.float64)

assert np.allclose(A, A.T), "Матрица не симметрична"

U = square_root(A)

z = solve_lower_triangular(U.T.conj(), y)
x = solve_upper_triangular(U, z)

x_real = np.real_if_close(x)

x_exact =  np.linalg.solve(A, y)

residual = np.linalg.norm(A @ x_real - y)

eigenvalues = np.linalg.eigvals(A)
is_positive_definite = np.all(eigenvalues > 0)

print(f"Все собственные значения положительны: {'Да, значит метод применим' if is_positive_definite else 'Нет, значит метод не применим'}")
print(f"Собственные значения: {eigenvalues.round(3)}")
print("-" * 40)

print("\nРазложение U:")
print(np.round(U, 4))
print("-" * 40)

print("\nПриближенное решение:")
print(f"x = {np.round(x_real, 6)}")
print(f"Норма невязки: {residual:.6f}")
print("\nТочное решение (Wolfram):")
print(f"x = {x_exact}")
print("-" * 40)

Все собственные значения положительны: Нет, значит метод не применим
Собственные значения: [-3.324  3.461  1.554]
----------------------------------------

Разложение U:
[[ 0.911 +0.j      2.3929+0.j     -1.8989+0.j    ]
 [ 0.    +0.j      0.    +2.6713j  0.    -2.0866j]
 [ 0.    +0.j      0.    +0.j      0.    +2.3853j]]
----------------------------------------

Приближенное решение:
x = [ 2.212476 -0.514307 -0.066378]
Норма невязки: 8.266829

Точное решение (Wolfram):
x = [-0.21874575  0.36470614 -0.12514427]
----------------------------------------


In [188]:
import numpy as np

def simple_iteration_with_steps(B, c, epsilon=0.001, max_iter=1000):
    n = len(c)
    x_prev = np.zeros(n)
    history = []
    
    B_norm = np.max(np.sum(np.abs(B), axis=0))
    spectral_radius = np.max(np.abs(np.linalg.eigvals(B)))
    
    print("Теоретические проверки:")
    print(f"1. Достаточное условие: ||B||_1 = {B_norm:.2f} {'< 1' if B_norm < 1 else '≥ 1'}")
    print(f"2. Необходимое условие: ρ(B) = {spectral_radius:.2f} {'< 1' if spectral_radius < 1 else '≥ 1'}")
    
    if B_norm >= 1 and spectral_radius >= 1:
        raise ValueError("Метод гарантированно расходится: оба условия нарушены")
    elif spectral_radius >= 1:
        print("ρ(B) ≥ 1. Метод может расходиться!")
    elif B_norm >= 1:
        print("||B||₁ ≥ 1. Сходимость не гарантируется, но возможна")
    else:
        print("Оба условия выполнены. Метод гарантированно сходится")
    
    print("-" * 40)
    
    for iter in range(max_iter):
        x_current = B @ x_prev + c
        diff = x_current - x_prev
        norm = np.max(np.abs(diff))
        
        history.append({
            "iteration": iter + 1,
            "x": x_current.copy(),
            "norm": norm,
            "spectral_radius": spectral_radius
        })
        
        print(f"Итерация {iter + 1}:")
        print(f"x = {np.round(x_current, 4)}")
        print(f"Δx: {norm:.4f}")
        print("-" * 40)
        
        if norm < epsilon:
            break
            
        x_prev = x_current.copy()
    else:
        print(f"Точность не достигнута за {max_iter} итераций")
    
    return history

# ----------------------------------------------------------
B = np.array([
    [0.00, 0.22, -0.11, 0.31],
    [0.38, 0.00, -0.12, 0.22],
    [0.11, 0.23, 0.00, -0.51],
    [0.17, -0.21, 0.31, 0.00]
], dtype=float)

c = np.array([2.7, -1.5, 1.2, -0.17], dtype=float)

history = simple_iteration_with_steps(B, c, epsilon=0.001)

final_solution = history[-1]["x"]
print("\nФинальное решение (x₁-x₄):")
for i, x in enumerate(final_solution.round(3), 1):
    print(f"x{i} = {x:.3f}")

Теоретические проверки:
1. Достаточное условие: ||B||_1 = 1.04 ≥ 1
2. Необходимое условие: ρ(B) = 0.46 < 1
||B||₁ ≥ 1. Сходимость не гарантируется, но возможна
----------------------------------------
Итерация 1:
x = [ 2.7  -1.5   1.2  -0.17]
Δx: 2.7000
----------------------------------------
Итерация 2:
x = [ 2.1853 -0.6554  1.2387  0.976 ]
Δx: 1.1460
----------------------------------------
Итерация 3:
x = [ 2.7221 -0.6035  0.7919  0.7231]
Δx: 0.5368
----------------------------------------
Итерация 4:
x = [ 2.7043 -0.4015  0.9918  0.665 ]
Δx: 0.2020
----------------------------------------
Итерация 5:
x = [ 2.7087 -0.4451  1.066   0.6815]
Δx: 0.0742
----------------------------------------
Итерация 6:
x = [ 2.6961 -0.4487  1.048   0.7144]
Δx: 0.0329
----------------------------------------
Итерация 7:
x = [ 2.7075 -0.4441  1.029   0.7074]
Δx: 0.0190
----------------------------------------
Итерация 8:
x = [ 2.7084 -0.439   1.0349  0.7025]
Δx: 0.0059
--------------------------------

In [192]:
import numpy as np

def seidel_method_v2(A, b, epsilon=0.001, max_iter=100):
    n = len(b)
    x = np.zeros(n)
    
    D = np.diag(np.diag(A))
    U = np.triu(A, 1)  
    L = np.tril(A, -1)  
    
    try:
        F = -np.linalg.inv(D + U) @ L
    except np.linalg.LinAlgError:
        raise ValueError("Матрица (D+U) вырождена")
    
    F_norm = np.max(np.sum(np.abs(F), axis=0))  # Норма ||F||₁
    spectral_radius = np.max(np.abs(np.linalg.eigvals(F)))
    
    print("Теоретические проверки:")
    print(f"1. Достаточное условие (||F||₁ < 1): {F_norm:.4f} {'✓' if F_norm < 1 else '✗'}")
    print(f"2. Необходимое условие (ρ(F) < 1): {spectral_radius:.4f} {'✓' if spectral_radius < 1 else '✗'}")
    
    if F_norm >= 1 and spectral_radius >= 1:
        print("Оба условия нарушены")
    elif F_norm >= 1:
        print("Сходимость не гарантируется (||F||₁ ≥ 1)")
    elif spectral_radius >= 1:
        print("Сходимость не гарантируется (ρ(F) ≥ 1)")
    else:
        print("Оба условия выполнены")
    print("=" * 50)
    
    for k in range(max_iter):
        x_old = x.copy()
        
        for i in reversed(range(n)):
            sum_L = np.dot(A[i,:i], x_old[:i])
            sum_U = np.dot(A[i,i+1:], x[i+1:])
            x[i] = (b[i] - sum_L - sum_U) / A[i,i]
        
        delta = np.max(np.abs(x - x_old))
        print(f"Итерация {k+1}:")
        print(f"x = {np.round(x, 4)}")
        print(f"Δx: {delta:.4f}")
        print("-" * 50)
        
        if delta < epsilon:
            break
    
    return np.round(x, 3)

A = np.array([
    [5.6,  2.7, -1.7],
    [3.4, -3.6, -6.7],
    [0.8,  1.3,  3.7]
], dtype=float)

b = np.array([1.9, -2.4, 1.2], dtype=float)

solution = seidel_method_v2(A, b, epsilon=0.001)

print("\nФинальное решение:")
for i, val in enumerate(solution, 1):
    print(f"x{i} = {val:.3f}")

Теоретические проверки:
1. Достаточное условие (||F||₁ < 1): 2.2781 ✗
2. Необходимое условие (ρ(F) < 1): 0.3174 ✓
Сходимость не гарантируется (||F||₁ ≥ 1)
Итерация 1:
x = [0.4073 0.0631 0.3243]
Δx: 0.4073
--------------------------------------------------
Итерация 2:
x = [0.0895 0.6529 0.2141]
Δx: 0.5899
--------------------------------------------------
Итерация 3:
x = [0.0679 0.6105 0.0756]
Δx: 0.1385
--------------------------------------------------
Итерация 4:
x = [0.1012 0.5537 0.0951]
Δx: 0.0568
--------------------------------------------------
Итерация 5:
x = [0.1013 0.5614 0.1079]
Δx: 0.0128
--------------------------------------------------
Итерация 6:
x = [0.098  0.5667 0.1051]
Δx: 0.0053
--------------------------------------------------
Итерация 7:
x = [0.0982 0.5656 0.104 ]
Δx: 0.0011
--------------------------------------------------
Итерация 8:
x = [0.0985 0.5651 0.1044]
Δx: 0.0005
--------------------------------------------------

Финальное решение:
x1 = 0.098
x2 = 0

In [204]:
import numpy as np

def scale_for_diagonal_dominance(A, b):
    n = len(b)
    A_scaled = A.copy()
    b_scaled = b.copy()
    has_dominance = True

    for i in range(n):
        row = A[i, :]
        diag = abs(row[i])
        row_sum = np.sum(np.abs(row)) - diag

        if diag <= row_sum:
            if diag < 1e-10:
                print(f"Внимание: Нулевой диагональный элемент в строке {i}!")
                has_dominance = False
            scale_factor = (row_sum + 1e-6) / diag
            A_scaled[i, :] *= scale_factor
            b_scaled[i] *= scale_factor
            
            new_diag = abs(A_scaled[i, i])
            new_row_sum = np.sum(np.abs(A_scaled[i, :])) - new_diag
            if new_diag <= new_row_sum:
                has_dominance = False

    if not has_dominance:
        print("Предупреждение: Не удалось достичь диагонального преобладания!")
    
    return A_scaled, b_scaled

def simple_iteration(A, b, x_exact, epsilon=1e-4, max_iter=1000):
    n = len(b)
    
    A_scaled, b_scaled = scale_for_diagonal_dominance(A, b)
    
    x = np.zeros(n, dtype=np.float64)
    D = np.diag(np.diag(A_scaled))
    try:
        inv_D = np.linalg.inv(D)
    except np.linalg.LinAlgError:
        print("Ошибка: Вырожденная матрица D!")
        return np.full(n, np.nan)
    
    L = np.tril(A_scaled, -1)
    U = np.triu(A_scaled, 1)
    B = -inv_D @ (L + U)
    c = inv_D @ b_scaled
    
    spectral_radius = max(abs(np.linalg.eigvals(B)))
    print(f"Спектральный радиус (простая итерация): {spectral_radius:.4f}")
    if spectral_radius >= 1:
        print("Метод может расходиться!")
    
    for k in range(max_iter):
        x_new = B @ x + c
        
        if np.isnan(x_new).any():
            print("Обнаружены NaN! Прерывание итераций.")
            return x_new
        
        residual = np.linalg.norm(b_scaled - A_scaled @ x_new)
        error = np.linalg.norm(x_new - x_exact)
        
        print(f"Iter {k+1}: x={np.round(x_new,4)}, невязка={residual:.4e}, ошибка={error:.4e}")
        x = x_new.copy()
        if residual < epsilon or error < 0.001:
            break
    return x

def seidel_method(A, b, x_exact, epsilon=1e-4, max_iter=1000):
    n = len(b)
    
    A_scaled, b_scaled = scale_for_diagonal_dominance(A, b)
    
    x = np.zeros(n, dtype=np.float64)
    
    for k in range(max_iter):
        x_old = x.copy()
        for i in range(n):
            if abs(A_scaled[i, i]) < 1e-10:
                print(f"Ошибка: Нулевой диагональный элемент в строке {i}!")
                return np.full(n, np.nan)
            
            sum1 = np.dot(A_scaled[i, :i], x[:i])
            sum2 = np.dot(A_scaled[i, i+1:], x[i+1:])
            x[i] = (b_scaled[i] - sum1 - sum2) / A_scaled[i, i]
        
        if np.isnan(x).any():
            print("Обнаружены NaN! Прерывание итераций.")
            return x
        
        residual = np.linalg.norm(b_scaled - A_scaled @ x)
        error = np.linalg.norm(x - x_exact)
        
        print(f"Iter {k+1}: x={np.round(x,4)}, невязка={residual:.4e}, ошибка={error:.4e}")
        if residual < epsilon or error < 0.001:
            break
    return x

def gradient_descent_with_checks(A, b, x_exact, epsilon=1e-4, max_iter=10000):
    n = len(b)
    x = np.zeros(n, dtype=np.float64)
    
    A_scaled, b_scaled = scale_for_diagonal_dominance(A, b)
    
    A_new = A_scaled.T @ A_scaled
    b_new = A_scaled.T @ b_scaled
    
    eigenvalues = np.linalg.eigvals(A_new)
    M = np.max(eigenvalues)
    m = np.min(eigenvalues)
    cond_number = M / m
    
    print("Теоретические параметры:")
    print(f"Спектральный радиус: {(M - m)/(M + m):.4f}")
    print(f"Число обусловленности: {cond_number:.2f}")
    print("=" * 50)
    
    for k in range(max_iter):
        r = b_new - A_new @ x
        Ar = A_new @ r
        dzeta = np.dot(r, r) / np.dot(r, Ar)
        x_new = x + dzeta * r
        
        residual = np.linalg.norm(b_scaled - A_scaled @ x_new)
        error = np.linalg.norm(x_new - x_exact)
        
        print(f"Итерация {k+1}:")
        print(f"x = {np.round(x_new, 4)}")
        print(f"Норма невязки: {residual:.4e}")
        print(f"Ошибка: {error:.4e}")
        print("-" * 50)
        
        x = x_new.copy()
        if residual < epsilon or error < 0.001:
            break
    return x

def gauss_solve(A, b):
    return np.linalg.solve(A, b)

A = np.array([
    [0.73, 1.24, -0.38, -1.43],
    [1.07, -0.77, 1.25, 0.66], 
    [1.56, 0.66, 1.44, -0.87],
    [0.75, 1.22, -0.83, 0.37]
], dtype=np.float64)

b = np.array([0.58, -0.66, 1.24, 0.92], dtype=np.float64)
epsilon = 1e-4

x_gauss = gauss_solve(A, b)

print("="*50 + "\nМетод простой итерации:\n" + "="*50)
x_simple = simple_iteration(A, b, x_gauss, epsilon)

print("\n" + "="*50 + "\nМетод Зейделя:\n" + "="*50)
x_seidel = seidel_method(A, b, x_gauss, epsilon)

print("\n" + "="*50 + "\nГрадиентный спуск:\n" + "="*50)
x_gradient = gradient_descent_with_checks(A, b, x_gauss, epsilon)

print("\n" + "="*50 + "\nСравнение решений:")
print(f"Гаусс:      {np.round(x_gauss, 4)}")
print(f"Простая ит: {np.round(x_simple, 4)} (Δ={np.linalg.norm(x_simple - x_gauss):.2e})")
print(f"Зейдель:    {np.round(x_seidel, 4)} (Δ={np.linalg.norm(x_seidel - x_gauss):.2e})")
print(f"Гр.спуск:   {np.round(x_gradient, 4)} (Δ={np.linalg.norm(x_gradient - x_gauss):.2e})")

Метод простой итерации:
Предупреждение: Не удалось достичь диагонального преобладания!
Спектральный радиус (простая итерация): 3.3661
Метод может расходиться!
Iter 1: x=[0.7945 0.8571 0.8611 2.4865], невязка=1.9474e+01, ошибка=2.9335e+00
Iter 2: x=[ 4.6576  5.4904  1.1098 -0.0186], невязка=7.8585e+01, ошибка=6.6191e+00
Iter 3: x=[ -7.9904   9.115   -6.7123 -22.5686], невязка=2.2253e+02, ошибка=2.6406e+01
Iter 4: x=[-62.3922 -40.4874  -8.2955 -26.429 ], невязка=8.6331e+02, ошибка=8.0019e+01
Iter 5: x=[  13.4776 -121.9639   71.0419  243.8474], невязка=2.7225e+03, ошибка=2.8223e+02
Iter 6: x=[722.6204 343.9255 189.485  536.6827], невязка=9.0570e+03, ошибка=9.8148e+02
Iter 7: x=[  566.5382  1772.6373  -615.3644 -2171.2484], невязка=3.3069e+04, ошибка=2.9247e+03
Iter 8: x=[-7583.8546 -2071.9137 -2737.1433 -8371.226 ], невязка=9.5561e+04, ошибка=1.1806e+04
Iter 9: x=[-14303.0339 -22156.497    4108.7149  16066.8019], невязка=3.9122e+05, ошибка=3.1154e+04
Iter 10: x=[ 71248.5954    566.7489  3

C:\Users\Данил\AppData\Local\Temp\ipykernel_3484\1914555054.py:62: RuntimeWarning: overflow encountered in matmul
  residual = np.linalg.norm(b_scaled - A_scaled @ x_new)
C:\Users\Данил\AppData\Local\Temp\ipykernel_3484\1914555054.py:62: RuntimeWarning: invalid value encountered in matmul
  residual = np.linalg.norm(b_scaled - A_scaled @ x_new)
C:\Users\Данил\AppData\Local\Temp\ipykernel_3484\1914555054.py:56: RuntimeWarning: overflow encountered in matmul
  x_new = B @ x + c
C:\Users\Данил\AppData\Local\Temp\ipykernel_3484\1914555054.py:56: RuntimeWarning: invalid value encountered in matmul
  x_new = B @ x + c
C:\Users\Данил\AppData\Local\Temp\ipykernel_3484\1914555054.py:93: RuntimeWarning: overflow encountered in matmul
  residual = np.linalg.norm(b_scaled - A_scaled @ x)
C:\Users\Данил\AppData\Local\Temp\ipykernel_3484\1914555054.py:93: RuntimeWarning: invalid value encountered in matmul
  residual = np.linalg.norm(b_scaled - A_scaled @ x)
C:\Users\Данил\AppData\Local\Temp\ipykern